In [1]:
import pandas as pd
import numpy as np

In [2]:
headlines = pd.read_csv('../data/processed/headlines_cl_sent_pol_rapi.csv')

In [3]:
headlines.drop_duplicates(subset = ['clean_hl_words'], inplace = True)

In [4]:
countries_freq = pd.read_csv('../data/processed/countries_freq_rapi.csv')

In [5]:
countries_freq

,Unnamed: 0,word,frequency,country
0,0,man,616,South Africa
1,1,sa,568,South Africa
2,2,kill,552,South Africa
3,3,watch,541,South Africa
4,4,rape,481,South Africa
...,...,...,...,...
110905,29351,rogers,1,India
110906,29352,pocketful,1,India
110907,29353,blasts,1,India
110908,29354,candlelight,1,India


In [6]:
headlines['time']= headlines.time.str.split('+', expand=True)[0]

In [7]:
df = headlines[['clean_hl_words', 'country', 'time']]

In [8]:
from datetime import datetime
df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

<ipython-input-8-6351246ca3f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))


In [9]:
df['clean_hl_words'] = df['clean_hl_words'].str.replace('{', '')
df['clean_hl_words'] = df['clean_hl_words'].str.replace('}', '')
df['clean_hl_words'] = df['clean_hl_words'].str.split(",")

<ipython-input-9-68623b5807f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_hl_words'] = df['clean_hl_words'].str.replace('{', '')
<ipython-input-9-68623b5807f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_hl_words'] = df['clean_hl_words'].str.replace('}', '')
<ipython-input-9-68623b5807f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
%%time
freq = df.set_index('time').groupby([pd.Grouper(freq = 'Y'), 'country']).sum()

In [11]:
print("There is " ,len(freq.reset_index()['time'].unique()), "year of data")

There is  17 year of data


In [12]:
freq = freq.reset_index()

In [18]:
freq

,time,country,clean_hl_words
0,2005-12-31,India,"{''aug'': 2, ' 'son'': 2, ' 'monster'': 2, ' '..."
1,2005-12-31,South Africa,"{''monroe'': 1, ' 'stand'': 1, ' 'night'': 1}"
2,2005-12-31,UK,"{''call'': 1, ' 'kill'': 3, ' 'parent'': 1, ''..."
3,2005-12-31,USA,"{''iowa'': 1, ' 'snatch'': 1, ' 'dead'': 2, ' ..."
4,2006-12-31,India,"{''ramsey'': 1, ' 'jun'': 1, ' 'die'': 2, ' 'j..."
...,...,...,...
61,2020-12-31,USA,"{''young'': 1, ' 'batter'': 1, ' 'chicago'': 1..."
62,2021-12-31,India,"{''leader'': 2, ' 'world'': 2, ' 'aspire'': 1,..."
63,2021-12-31,South Africa,"{''reinhart'': 1, ' 'nation'': 1, ' 'bank'': 1..."
64,2021-12-31,UK,"{''tills'': 1, ' 'young'': 1, ' 'elderly'': 2,..."


In [17]:
import collections
freq['clean_hl_words'] = freq['clean_hl_words'].apply(lambda x: collections.Counter(x))

In [ ]:
def prop(list):
    

In [85]:
test = [1,1,1,0]
a = collections.Counter(test)


dict_values([3, 1])

In [92]:
def get_proportional_frequency(dict):
    return {k: 100*(v/ sum([v for k,v in dict.items()])) for k,v in dict.items()}

In [94]:
freq['clean_hl_words_prop']= freq['clean_hl_words'].apply(lambda x: get_proportional_frequency(x))

In [95]:
import json
df2 = freq.join(freq.clean_hl_words_prop.apply(pd.Series), how = 'left').melt(id_vars=['time', 'country', 'clean_hl_words', 'clean_hl_words_prop'])

In [ ]:
df2.drop(columns = ['clean_hl_words', 'clean_hl_words_prop'], inplace = True)

In [99]:
df2.dropna

,time,country,variable,value
0,2005-12-31,India,'aug',0.450450
1,2005-12-31,South Africa,'aug',NaN
2,2005-12-31,UK,'aug',NaN
3,2005-12-31,USA,'aug',NaN
4,2006-12-31,India,'aug',NaN
...,...,...,...,...
5397343,2020-12-31,USA,'mynatt',NaN
5397344,2021-12-31,India,'mynatt',NaN
5397345,2021-12-31,South Africa,'mynatt',NaN
5397346,2021-12-31,UK,'mynatt',NaN


In [100]:
df2.columns = ['time', 'country', 'word', 'prop_frequency']

In [102]:
df2['word'] = df2['word'].str.strip("'").str.strip("'")
#df['word'] 

In [104]:
df = df.merge(df2, on = ['time', 'country', 'word'])

In [22]:
df.drop(columns = ['clean_hl_words'], inplace = True)

In [175]:
df.drop(columns = ['clean_hl_words', 'clean_hl_freq'], inplace = True)

In [26]:
df

,time,country,word,frequency
0,2005-12-31,India,'aug',2.0
1,2005-12-31,South Africa,'aug',NaN
2,2005-12-31,UK,'aug',NaN
3,2005-12-31,USA,'aug',NaN
4,2006-12-31,India,'aug',NaN
...,...,...,...,...
5397343,2020-12-31,USA,'mynatt',NaN
5397344,2021-12-31,India,'mynatt',NaN
5397345,2021-12-31,South Africa,'mynatt',NaN
5397346,2021-12-31,UK,'mynatt',NaN


In [39]:
countries_freq.sort_values(by='frequency', ascending = False).head(20)

,Unnamed: 0,word,frequency,country
10432,0,year,7656,USA
10433,1,new,7171,USA
10434,2,say,6755,USA
10435,3,man,6466,USA
10436,4,old,5386,USA
49863,0,man,4996,UK
49864,1,year,4563,UK
81554,0,india,4106,India
81555,1,year,3929,India
10437,5,gaga,3858,USA


In [64]:
df['word'] = df['word'].str.strip("'").str.strip("'")
df['word'] 

In [107]:
df.to_csv('../data/processed/country_time_freq_rapi.csv')

In [106]:
df[(df['word'] == 'kill')& (df['country']=='India')]

,time,country,word,frequency,prop_frequency
1518,2005-12-31,India,kill,4.0,0.900901
1522,2006-12-31,India,kill,1.0,0.189394
1526,2007-12-31,India,kill,3.0,0.224551
1529,2008-12-31,India,kill,8.0,0.181324
1532,2009-12-31,India,kill,14.0,0.186517
1536,2010-12-31,India,kill,17.0,0.172047
1540,2011-12-31,India,kill,25.0,0.173503
1544,2012-12-31,India,kill,34.0,0.154223
1548,2013-12-31,India,kill,41.0,0.139617
1552,2014-12-31,India,kill,64.0,0.142838


In [ ]:
df.groupby(['country', 'word']).sum().sort_values(by = 'frequency')